### **Introduction to Arachne**
Arachne is a Python package for graph analysis that is built as an extension to Arkouda, a Python package for analysis on tabular data, akin to NumPy and Pandas. In this notebook we will show examples on how to run each algorithm that has been implemented on different types of graphs: undirected, directed, and property.

In [ ]:
import arkouda as ak
import arachne as ar
import networkx as nx
import os

In [ ]:
# NOTE: Make sure to change the server name to whatever is applicable in your environment. If running locally, then use only ak.connect().
ak.connect("n116", 5555)

### **Graph Generation and Loading**
Graphs can be built from existing data or generated with our suite of random graph generators. The preferred way to load a graph into memory is from Arkouda arrays, however we provide a method to read a graph in from matrix market format, or to generate them from existing Arkouda data.

In [ ]:
# Read in a graph from a matrix market file.
absolute_path_to_karate = os.path.abspath("data/karate.mtx")
karate = ar.read_matrix_market_file(absolute_path_to_karate)

In [ ]:
# Generate a random RMAT graph.
random_graph = ar.rmat(20, create_using=ar.Graph)

In [ ]:
n = 1_000
m = 1_000_000
k = 2

In [ ]:
src_array = ak.randint(0, n, m, dtype=ak.dtype('int64'), seed=2)
dst_array = ak.randint(0, n, m, dtype=ak.dtype('int64'), seed=4)
int_array = ak.randint(-1, k, m, dtype=ak.dtype('int64'), seed=6)
uint_array = ak.randint(0, k, m, dtype=ak.dtype('uint64'), seed=8)
real_array = ak.randint(0, k, m, dtype=ak.dtype('float64'), seed=10)
bool_array = ak.randint(0, k, m, dtype=ak.dtype('bool'), seed=12)
strings_array = ak.random_strings_uniform(0, k, m, characters="abcdefghijklmonpqrstuvwxyz", seed=14)
categorical_array = ak.Categorical(ak.random_strings_uniform(0, k, m, characters="abcdefghijklmonpqrstuvwxyz", seed=14))

In [ ]:
prop_graph = ar.PropGraph()

In [ ]:
test_edge_dict = {
    "src":src_array,
    "dst":dst_array,
    "data1":int_array,
    "data2":uint_array,
    "data3":real_array,
    "data4":bool_array,
    "data5":strings_array,
    "data6":categorical_array
}
test_edge_df = ak.DataFrame(test_edge_dict)

In [ ]:
prop_graph.load_edge_attributes(test_edge_df, source_column="src", destination_column="dst", relationship_columns=["data5", "data1"])

In [ ]:
prop_graph.get_edge_attributes()

In [ ]:
prop_graph.get_edge_relationships()

In [ ]:
print(type(prop_graph.edge_attributes["data5"]))

In [ ]:
# prop_graph.add_edges_from(src_array,dst_array)

In [ ]:
m = len(prop_graph)
k = 2

In [ ]:
int_array = ak.randint(-1, k, m, dtype=ak.dtype('int64'), seed=6)
uint_array = ak.randint(0, k, m, dtype=ak.dtype('uint64'), seed=8)
real_array = ak.randint(0, k, m, dtype=ak.dtype('float64'), seed=10)
bool_array = ak.randint(0, k, m, dtype=ak.dtype('bool'), seed=12)
strings_array = ak.random_strings_uniform(0, k, m, characters="abcdefghijklmonpqrstuvwxyz", seed=14)
categorical_array = ak.Categorical(ak.random_strings_uniform(0, k, m, characters="abcdefghijklmonpqrstuvwxyz", seed=14))

In [ ]:
test_node_dict = {
    "nodes":prop_graph.nodes(),
    "data1":int_array,
    "data2":uint_array,
    "data3":real_array,
    "data4":bool_array,
    "data5":strings_array,
    "data6":categorical_array
}
test_node_df = ak.DataFrame(test_node_dict)

In [ ]:
prop_graph.load_node_attributes(test_node_df, node_column="nodes", label_columns=["data5", "data2"])

In [ ]:
prop_graph.get_node_attributes()

In [ ]:
prop_graph.get_node_labels()

In [ ]:
def node_filter(node_attributes):
    return node_attributes["data2"] == 0

In [ ]:
def edge_filter(edge_attributes):
    return edge_attributes["data1"] > -1

In [ ]:
subgraph_nodes = prop_graph.subgraph_view(filter_node=node_filter)

In [ ]:
subgraph_edges = prop_graph.subgraph_view(filter_edge=edge_filter)

In [ ]:
subgraph_together = prop_graph.subgraph_view(filter_node=node_filter, filter_edge=edge_filter)

In [ ]:
print(f"Subgraph generated with edge size: {subgraph_nodes.size()}")

In [ ]:
print(f"Subgraph generated with edge size: {subgraph_edges.size()}")

In [ ]:
print(f"Subgraph generated with edge size: {subgraph_together.size()}")